In [1]:
import pandas as pd

DATA_PATH = '../data/'
filename  = 'estrepublicain_annee_1999.csv'

df = pd.read_csv(DATA_PATH + filename)
# df = df[0:1000]

# sampler 

df = df.sample(frac= 0.5)
df.reset_index(inplace = True, drop = True)
print(df.shape)

(15120, 1)


In [2]:
df
print("= dimensions")
print(df.shape)
print("= 5 premieres rangés")
print(df.head())
print("= Noms des colonnes")
print(df.columns)


= dimensions
(15120, 1)
= 5 premieres rangés
                                                text
0  Contrat éducatif local : le maire informe le c...
1  On reproche aux deux hommes de s'être livré à ...
2  1. El Mezoued (Maroc) en 44'57 ; 2. Mendes (Fa...
3  Les inscriptions à l'école municipale de musiq...
4  D'autant que dans le cas précis, il est diffic...
= Noms des colonnes
Index(['text'], dtype='object')


In [3]:
# Changer le nom de la colonne
df.columns = ['paragraphe']
df.columns

Index(['paragraphe'], dtype='object')

In [4]:
# puis renommer colonne en 'text'

df.columns = ['text']
df.columns

Index(['text'], dtype='object')

In [5]:
# Ne prendre que les 1000 premieres rangées

# print("avant: {}".format(df.shape))
# df = df[0:1000]

# print("apres: {}".format(df.shape))



In [6]:
# Tokenization
from nltk import word_tokenize

# La phrase
sentence = "les médiateurs confirment « l'absence de solution parfaite ». "

tokens = word_tokenize(sentence)

print(tokens)
tokens_on_space = sentence.split(' ')
print(tokens_on_space)


['les', 'médiateurs', 'confirment', '«', "l'absence", 'de', 'solution', 'parfaite', '»', '.']
['les', 'médiateurs', 'confirment', '«', "l'absence", 'de', 'solution', 'parfaite', '».', '']


In [7]:
# ponctuation

import string

print("Liste original de signe de ponctuations")
print("\t{}".format(string.punctuation))

sentence = "les médiateurs confirment « l'absence de solution parfaite ». "
punctuation_chars = string.punctuation + "«»"

#  Construction d'un dict { '~':' ', '$': ' ', ... }
dict_ponctuation = {}
for k in punctuation_chars:
    dict_ponctuation[k] = ' '

print(dict_ponctuation)

# # L'operateur de translation
translator = str.maketrans(dict_ponctuation)

new_sentence = sentence.translate(translator)
print(sentence)
print(new_sentence)


Liste original de signe de ponctuations
	!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{'!': ' ', '"': ' ', '#': ' ', '$': ' ', '%': ' ', '&': ' ', "'": ' ', '(': ' ', ')': ' ', '*': ' ', '+': ' ', ',': ' ', '-': ' ', '.': ' ', '/': ' ', ':': ' ', ';': ' ', '<': ' ', '=': ' ', '>': ' ', '?': ' ', '@': ' ', '[': ' ', '\\': ' ', ']': ' ', '^': ' ', '_': ' ', '`': ' ', '{': ' ', '|': ' ', '}': ' ', '~': ' ', '«': ' ', '»': ' '}
les médiateurs confirment « l'absence de solution parfaite ». 
les médiateurs confirment   l absence de solution parfaite    


In [40]:
# stopwords
tokens = word_tokenize(new_sentence)
print(tokens)

from nltk.corpus import stopwords


# print("=== stopwords - français:")
# print(sorted(stopwords.words('french')))

# stopwords de nltk
list_stopwords = stopwords.words('french') + ['les', 'de']

# alternative
from stop_words import get_stop_words
list_stopwords = get_stop_words('fr')



# # equivalent:
# stopwords = stopwords.words('french')
# stopwords.append('les')
# stopwords.append('de')


# 
tokens_sans_stopwords = [w for w in tokens if (w not in list_stopwords) ]

## Equivalent 
tokens_sans_stopwords = []
for w in tokens:
    if w not in list_stopwords:
        tokens_sans_stopwords.append(w)



print(tokens_sans_stopwords)


['les', 'médiateurs', 'confirment', 'l', 'absence', 'de', 'solution', 'parfaite']
['médiateurs', 'confirment', 'absence', 'solution', 'parfaite']


In [9]:
# Enlevons la ponctuation

df['text_no_punctuation'] = df.text.apply(lambda 
    r : ( r.translate(translator) ) 
)
df.head()



,text,text_no_punctuation
0,Contrat éducatif local : le maire informe le c...,Contrat éducatif local le maire informe le c...
1,On reproche aux deux hommes de s'être livré à ...,On reproche aux deux hommes de s être livré à ...
2,1. El Mezoued (Maroc) en 44'57 ; 2. Mendes (Fa...,1 El Mezoued Maroc en 44 57 2 Mendes Fa...
3,Les inscriptions à l'école municipale de musiq...,Les inscriptions à l école municipale de musiq...
4,"D'autant que dans le cas précis, il est diffic...",D autant que dans le cas précis il est diffic...


In [10]:
# Tokenizer

df['tokens_all']  = df.text_no_punctuation.apply(
    lambda r : word_tokenize(r.lower())
)

df.head()

,text,text_no_punctuation,tokens_all
0,Contrat éducatif local : le maire informe le c...,Contrat éducatif local le maire informe le c...,"[contrat, éducatif, local, le, maire, informe,..."
1,On reproche aux deux hommes de s'être livré à ...,On reproche aux deux hommes de s être livré à ...,"[on, reproche, aux, deux, hommes, de, s, être,..."
2,1. El Mezoued (Maroc) en 44'57 ; 2. Mendes (Fa...,1 El Mezoued Maroc en 44 57 2 Mendes Fa...,"[1, el, mezoued, maroc, en, 44, 57, 2, mendes,..."
3,Les inscriptions à l'école municipale de musiq...,Les inscriptions à l école municipale de musiq...,"[les, inscriptions, à, l, école, municipale, d..."
4,"D'autant que dans le cas précis, il est diffic...",D autant que dans le cas précis il est diffic...,"[d, autant, que, dans, le, cas, précis, il, es..."


In [11]:
# remove stopwords

def remove_stopword(tokens):
     return [w for w in tokens if (w not in list_stopwords) ]

# Verifier que ca marche
remove_stopword(tokens)

# appliquer a la dataframe

df['tokens'] = df.tokens_all.apply(
    lambda tks : remove_stopword(tks) 
)

df.head()

,text,text_no_punctuation,tokens_all,tokens
0,Contrat éducatif local : le maire informe le c...,Contrat éducatif local le maire informe le c...,"[contrat, éducatif, local, le, maire, informe,...","[contrat, éducatif, local, maire, informe, con..."
1,On reproche aux deux hommes de s'être livré à ...,On reproche aux deux hommes de s être livré à ...,"[on, reproche, aux, deux, hommes, de, s, être,...","[reproche, deux, hommes, être, livré, opératio..."
2,1. El Mezoued (Maroc) en 44'57 ; 2. Mendes (Fa...,1 El Mezoued Maroc en 44 57 2 Mendes Fa...,"[1, el, mezoued, maroc, en, 44, 57, 2, mendes,...","[1, el, mezoued, maroc, 44, 57, 2, mendes, fav..."
3,Les inscriptions à l'école municipale de musiq...,Les inscriptions à l école municipale de musiq...,"[les, inscriptions, à, l, école, municipale, d...","[inscriptions, école, municipale, musique, lie..."
4,"D'autant que dans le cas précis, il est diffic...",D autant que dans le cas précis il est diffic...,"[d, autant, que, dans, le, cas, précis, il, es...","[autant, cas, précis, difficile, établir, a, e..."


In [12]:
# enlever les colonnes intermediaires

print(df.columns)

df = df[['text', 'tokens']]

print(df.columns)


Index(['text', 'text_no_punctuation', 'tokens_all', 'tokens'], dtype='object')
Index(['text', 'tokens'], dtype='object')


In [13]:
# nombre de tokens par rangée
pd.options.mode.chained_assignment = None
df['token_count'] = df.tokens.apply( lambda r : len(r) )

print(df.head())

# repartition du nombre de tokens
df.token_count.describe()


                                                text  \
0  Contrat éducatif local : le maire informe le c...   
1  On reproche aux deux hommes de s'être livré à ...   
2  1. El Mezoued (Maroc) en 44'57 ; 2. Mendes (Fa...   
3  Les inscriptions à l'école municipale de musiq...   
4  D'autant que dans le cas précis, il est diffic...   

                                              tokens  token_count  
0  [contrat, éducatif, local, maire, informe, con...           72  
1  [reproche, deux, hommes, être, livré, opératio...           53  
2  [1, el, mezoued, maroc, 44, 57, 2, mendes, fav...          278  
3  [inscriptions, école, municipale, musique, lie...           68  
4  [autant, cas, précis, difficile, établir, a, e...           53  


count    15120.000000
mean        80.645238
std         35.202579
min         44.000000
25%         64.000000
50%         73.000000
75%         86.000000
max       1011.000000
Name: token_count, dtype: float64

In [14]:
# quel document a 955 tokens?

condition = (df.token_count == 955)
print(df[condition].text.values)




[ "161. Brisset (CSA BA 133) ; 162. Tascone ; 163. Leroy (SNCF) ; 164. Cassin (Tomblaine) ; 165. Aubert (Frouard) ; 166. Pascaot ; 167. Lagrange (Messein) ; 168. Vermandé (Messein) ; 169. Rouyer (Commercy) ; 170. Chrisment (Nancy) ; 171. Lepeltier (Ecrouves) ; 172. Jacquot ; 173. Delery ; 174. Billy ; 175. Lamy ; 176. Charpentier (Ludres) ; 177. Tousch (Gondrecourt) ; 178. Carbillet (La Poste) ; 179. Brunot (Philips) ; 180. Benyoussef ; 181. Starosse (Allamps) ; 182. Seiler (Vandoeuvre) ; 183.Discristofano ; 184. Fossard (Vandoeuvre) ; 185. Boyer ; 186. Leroy ; 187. Carreiras (Golbey) ; 188. Evrard ; 189. Vagne (CSA BA 133) ; 190. Georges (Tomblaine) ; 191. Aubert (Maidières) ; 192. Leroux ; 193. Geoffroy ; 194. Clausse (Chaudenay) ; 195. Deuze (Villers) ; 196. P. Knapek (Ecrouves) ; 197. F. Kanpek (Toul) ; 198. Noël ; 199. Maire (Varangéville) ; 200. Mercky ; 201. Gaumer ; 202. Vincent (Sommerviller) ; 203. Socha (Aingeray) ; 204. Boulot (Blénod) ; 205. Houin (Amicale CHU) ; 206. Borg

In [15]:
# quel index
print(df[condition].index)

# 
condition = (df.token_count < 800)
print(df[condition].shape)


Int64Index([13863], dtype='int64')
(15116, 3)


In [16]:
# enlever le paragraphe le plus long

condition_filtrage = df.token_count < 955
print(df[condition_filtrage].shape)

df = df[condition_filtrage]
print(df[condition_filtrage].shape)

print(df.token_count.describe())


(15118, 3)
(15118, 3)
count    15118.000000
mean        80.525863
std         33.638400
min         44.000000
25%         64.000000
50%         73.000000
75%         86.000000
max        949.000000
Name: token_count, dtype: float64


/home/alexis/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  import sys


In [17]:
# Gensim - Vocabulaire

from gensim import corpora, models
dictionary  = corpora.Dictionary(df.tokens)
print(dictionary)


Dictionary(81995 unique tokens: ['contrat', 'éducatif', 'local', 'maire', 'informe']...)


In [18]:
# corpus_gensim

df['corpus_gensim'] = df.tokens.apply(lambda d : dictionary.doc2bow(d))

print(df[['text','corpus_gensim']] .head().values)


[[ "Contrat éducatif local : le maire informe le conseil municipal, qu'il a été contacté par les services de la Direction départementale de la jeunesse et loisirs, qui propose à la commune, sur la base des actions déjà menées pour les enfants et les adolescents sur la commune, d'élaborer un projet de contrat éducatif local. Ce contrat propose de renforcer le partenariat entre les associations locales, l'Etat, Jeunesse et sports, Education nationale et la commune, d'améliorer la coordination des interventions vers les jeunes, afin de favoriser la réussite scolaire, la citoyenneté, l'autonomie, la responsabilité et l'intégration des enfants et des jeunes. Enfin, il permet l'attribution de moyens supplémentaires. Après débat, le conseil municipal, à l'unanimité, autorise le maire à poursuivre le montage de ce projet avec la DDJS."
  list([(0, 3), (1, 2), (2, 2), (3, 2), (4, 1), (5, 2), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 2), (15, 3), (16, 1), (17, 1), 

In [20]:
corpus_gensim = [c for c in df.corpus_gensim ]


In [21]:
num_topics= 40

# Le model LDA
lda = models.LdaModel(corpus_gensim,
    id2word      = dictionary,
    num_topics   = num_topics,
    alpha        = 'asymmetric',
    eta          = 'auto',
    passes       = 2,
    iterations   = 20
)




In [33]:
for t in lda.show_topics(num_topics=num_topics, formatted=True, log = False):
    print("\n=== topic #{}".format(t[0]))
    print(t[1].replace('*', ': ').replace(' +',', ').replace('"',''))



=== topic #0
0.021: a,  0.009: cette,  0.005: plus,  0.005: jeunes,  0.004: être,  0.004: travail,  0.004: ans,  0.004: année,  0.004: depuis,  0.004: personnes

=== topic #1
0.007: pêche,  0.006: plus,  0.006: a,  0.003: poissons,  0.003: eau,  0.003: ainsi,  0.003: démonstrations,  0.003: deux,  0.002: année,  0.002: cette

=== topic #2
0.022: ils,  0.011: a,  0.009: plus,  0.008: leurs,  0.008: tous,  0.008: enfants,  0.008: tout,  0.005: comme,  0.005: faire,  0.005: deux

=== topic #3
0.008: deux,  0.008: après,  0.007: tête,  0.007: plus,  0.006: coup,  0.006: but,  0.005: jeu,  0.005: a,  0.005: minutes,  0.005: tir

=== topic #4
0.027: a,  0.012: plus,  0.008: tout,  0.008: cette,  0.007: bien,  0.007: équipe,  0.006: club,  0.006: deux,  0.006: fait,  0.005: très

=== topic #5
0.019: a,  0.011: cette,  0.007: plus,  0.005: ville,  0.005: saint,  0.004: fête,  0.004: sous,  0.004: village,  0.004: public,  0.004: aussi

=== topic #6
0.026: f,  0.021: 000,  0.016: a,  0.009: tr

'0.006*"attaché" + 0.005*"a" + 0.004*"becker" + 0.003*"hugues" + 0.003*"vues" + 0.003*"spinalienne" + 0.003*"pape" + 0.003*"dédé" + 0.003*"van" + 0.003*"complexes"'

In [ ]:
# LSA

tfidf = models.TfidfModel(corpus_gensim)

corpus_tfidf = tfidf[corpus_gensim]

lsi = models.LsiModel(corpus_tfidf , id2word=dictionary, num_topics=num_topics)
for t in lsi.show_topics(num_topics=num_topics, formatted=True, log = False):
    print("\n=== topic #{}".format(t[0]))
    print(t[1].replace('*', ': ').replace(' +',', ').replace('"',''))
